In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
from scipy.stats import gamma
import numpy as np
import c3aidatalake

In [9]:
all_fips = pd.read_csv('all_counties_fips.csv')
all_fips

,fips_code,county,state,c3_data_id
0,1001.0,Autauga,Alabama,Autauga_Alabama_UnitedStates
1,1003.0,Baldwin,Alabama,Baldwin_Alabama_UnitedStates
2,1005.0,Barbour,Alabama,Barbour_Alabama_UnitedStates
3,1007.0,Bibb,Alabama,Bibb_Alabama_UnitedStates
4,1009.0,Blount,Alabama,Blount_Alabama_UnitedStates
...,...,...,...,...
3133,56037.0,Sweetwater,Wyoming,Sweetwater_Wyoming_UnitedStates
3134,56039.0,Teton,Wyoming,Teton_Wyoming_UnitedStates
3135,56041.0,Uinta,Wyoming,Uinta_Wyoming_UnitedStates
3136,56043.0,Washakie,Wyoming,Washakie_Wyoming_UnitedStates


In [7]:
today = pd.Timestamp.now().strftime("%Y-%m-%d")

all_counties = c3aidatalake.evalmetrics(
    "outbreaklocation",
    {
        "spec" : {
            "ids" : list(all_counties['c3_data_id']),
            "expressions" : ["JHU_ConfirmedCases"],
            "start" : "2020-02-24",
            "end" : today,
            "interval" : "MONTH",
        }
    },
    get_all = True
)

all_counties

,dates,Blount_Alabama_UnitedStates.JHU_ConfirmedCases.data,Blount_Alabama_UnitedStates.JHU_ConfirmedCases.missing,Butler_Alabama_UnitedStates.JHU_ConfirmedCases.data,Butler_Alabama_UnitedStates.JHU_ConfirmedCases.missing,Chambers_Alabama_UnitedStates.JHU_ConfirmedCases.data,Chambers_Alabama_UnitedStates.JHU_ConfirmedCases.missing,Autauga_Alabama_UnitedStates.JHU_ConfirmedCases.data,Autauga_Alabama_UnitedStates.JHU_ConfirmedCases.missing,Bibb_Alabama_UnitedStates.JHU_ConfirmedCases.data,...,Lincoln_Wyoming_UnitedStates.JHU_ConfirmedCases.data,Lincoln_Wyoming_UnitedStates.JHU_ConfirmedCases.missing,Natrona_Wyoming_UnitedStates.JHU_ConfirmedCases.data,Natrona_Wyoming_UnitedStates.JHU_ConfirmedCases.missing,Sublette_Wyoming_UnitedStates.JHU_ConfirmedCases.data,Sublette_Wyoming_UnitedStates.JHU_ConfirmedCases.missing,Washakie_Wyoming_UnitedStates.JHU_ConfirmedCases.data,Washakie_Wyoming_UnitedStates.JHU_ConfirmedCases.missing,Weston_Wyoming_UnitedStates.JHU_ConfirmedCases.data,Weston_Wyoming_UnitedStates.JHU_ConfirmedCases.missing
0,2020-02-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,2020-03-01,5,0,1,0,36,0,7,0,3,...,0,0,12,0,1,0,1,0,0,0
0,2020-04-01,37,0,53,0,295,0,44,0,42,...,9,0,49,0,3,0,8,0,0,0
0,2020-05-01,62,0,411,0,355,0,221,0,75,...,15,0,77,0,3,0,36,0,1,0
0,2020-06-01,204,0,604,0,576,0,530,0,167,...,23,0,121,0,6,0,39,0,2,0
0,2020-07-01,685,0,739,0,806,0,972,0,355,...,89,0,215,0,34,0,48,0,5,0
0,2020-08-01,1034,0,837,0,873,0,1348,0,535,...,112,0,246,0,48,0,108,0,11,0
0,2020-09-01,1629,0,917,0,1179,0,1791,0,664,...,208,0,603,0,116,0,116,0,39,0
0,2020-10-01,2074,0,1022,0,1389,0,2159,0,873,...,314,0,1586,0,160,0,142,0,163,0
0,2020-11-01,2549,0,1087,0,1570,0,2456,0,986,...,479,0,3058,0,260,0,208,0,339,0


In [30]:
monthly_rural_cases.to_csv('./all_counties_model/jhu_all_counties_raw_cases.csv', index = False)

In [10]:
all_counties = monthly_rural_cases.copy()

In [11]:
months = all_counties.dates.dt.month_name()
all_counties['month'] = months

In [13]:
all_cases = all_counties.set_index('month').T
all_cases = all_cases.reset_index()
monthly_cases = all_cases[all_cases['index'].str.endswith(".data")]

In [14]:
monthly_cases 

month,index,February,March,April,May,June,July,August,September,October,November
1,Blount_Alabama_UnitedStates.JHU_ConfirmedCases...,0,5,37,62,204,685,1034,1629,2074,2549
3,Butler_Alabama_UnitedStates.JHU_ConfirmedCases...,0,1,53,411,604,739,837,917,1022,1087
5,Chambers_Alabama_UnitedStates.JHU_ConfirmedCas...,0,36,295,355,576,806,873,1179,1389,1570
7,Autauga_Alabama_UnitedStates.JHU_ConfirmedCase...,0,7,44,221,530,972,1348,1791,2159,2456
9,Bibb_Alabama_UnitedStates.JHU_ConfirmedCases.data,0,3,42,75,167,355,535,664,873,986
...,...,...,...,...,...,...,...,...,...,...,...
6275,Lincoln_Wyoming_UnitedStates.JHU_ConfirmedCase...,0,0,9,15,23,89,112,208,314,479
6277,Natrona_Wyoming_UnitedStates.JHU_ConfirmedCase...,0,12,49,77,121,215,246,603,1586,3058
6279,Sublette_Wyoming_UnitedStates.JHU_ConfirmedCas...,0,1,3,3,6,34,48,116,160,260
6281,Washakie_Wyoming_UnitedStates.JHU_ConfirmedCas...,0,1,8,36,39,48,108,116,142,208


In [15]:
#getting parent_id from id col
id_names = monthly_cases['index'].tolist()
split_names = [str(id_names[x]).replace(".JHU_ConfirmedCases.data", "")for x in range(len(id_names))]
monthly_cases['index'] = split_names
monthly_cases 

<ipython-input-15-15d289a64a30>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_cases['index'] = split_names


month,index,February,March,April,May,June,July,August,September,October,November
1,Blount_Alabama_UnitedStates,0,5,37,62,204,685,1034,1629,2074,2549
3,Butler_Alabama_UnitedStates,0,1,53,411,604,739,837,917,1022,1087
5,Chambers_Alabama_UnitedStates,0,36,295,355,576,806,873,1179,1389,1570
7,Autauga_Alabama_UnitedStates,0,7,44,221,530,972,1348,1791,2159,2456
9,Bibb_Alabama_UnitedStates,0,3,42,75,167,355,535,664,873,986
...,...,...,...,...,...,...,...,...,...,...,...
6275,Lincoln_Wyoming_UnitedStates,0,0,9,15,23,89,112,208,314,479
6277,Natrona_Wyoming_UnitedStates,0,12,49,77,121,215,246,603,1586,3058
6279,Sublette_Wyoming_UnitedStates,0,1,3,3,6,34,48,116,160,260
6281,Washakie_Wyoming_UnitedStates,0,1,8,36,39,48,108,116,142,208


In [20]:
mcases = all_fips.merge(monthly_cases, left_on='c3_data_id', right_on = "index")
mcases

,fips_code,county,state,c3_data_id,index,February,March,April,May,June,July,August,September,October,November
0,1001.0,Autauga,Alabama,Autauga_Alabama_UnitedStates,Autauga_Alabama_UnitedStates,0,7,44,221,530,972,1348,1791,2159,2456
1,1003.0,Baldwin,Alabama,Baldwin_Alabama_UnitedStates,Baldwin_Alabama_UnitedStates,0,19,174,290,663,3056,4422,5640,6940,7646
2,1005.0,Barbour,Alabama,Barbour_Alabama_UnitedStates,Barbour_Alabama_UnitedStates,0,0,39,167,322,550,628,896,1060,1128
3,1007.0,Bibb,Alabama,Bibb_Alabama_UnitedStates,Bibb_Alabama_UnitedStates,0,3,42,75,167,355,535,664,873,986
4,1009.0,Blount,Alabama,Blount_Alabama_UnitedStates,Blount_Alabama_UnitedStates,0,5,37,62,204,685,1034,1629,2074,2549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3133,56037.0,Sweetwater,Wyoming,Sweetwater_Wyoming_UnitedStates,Sweetwater_Wyoming_UnitedStates,0,2,18,30,90,240,302,339,488,1024
3134,56039.0,Teton,Wyoming,Teton_Wyoming_UnitedStates,Teton_Wyoming_UnitedStates,0,20,96,100,129,335,430,577,802,1156
3135,56041.0,Uinta,Wyoming,Uinta_Wyoming_UnitedStates,Uinta_Wyoming_UnitedStates,0,0,8,12,176,254,298,357,484,719
3136,56043.0,Washakie,Wyoming,Washakie_Wyoming_UnitedStates,Washakie_Wyoming_UnitedStates,0,1,8,36,39,48,108,116,142,208


# Cumulative Monthly Cases

In [28]:
cumulative_cases = mcases.copy().drop(columns = ['county', 'state', 'c3_data_id', 'index'])
cumulative_cases

,fips_code,February,March,April,May,June,July,August,September,October,November
0,1001.0,0,7,44,221,530,972,1348,1791,2159,2456
1,1003.0,0,19,174,290,663,3056,4422,5640,6940,7646
2,1005.0,0,0,39,167,322,550,628,896,1060,1128
3,1007.0,0,3,42,75,167,355,535,664,873,986
4,1009.0,0,5,37,62,204,685,1034,1629,2074,2549
...,...,...,...,...,...,...,...,...,...,...,...
3133,56037.0,0,2,18,30,90,240,302,339,488,1024
3134,56039.0,0,20,96,100,129,335,430,577,802,1156
3135,56041.0,0,0,8,12,176,254,298,357,484,719
3136,56043.0,0,1,8,36,39,48,108,116,142,208


## New Monthly Cases

In [25]:
df = mcases.copy().drop(columns = ['county', 'state', 'c3_data_id', 'index'])
df

,fips_code,February,March,April,May,June,July,August,September,October,November
0,1001.0,0,7,44,221,530,972,1348,1791,2159,2456
1,1003.0,0,19,174,290,663,3056,4422,5640,6940,7646
2,1005.0,0,0,39,167,322,550,628,896,1060,1128
3,1007.0,0,3,42,75,167,355,535,664,873,986
4,1009.0,0,5,37,62,204,685,1034,1629,2074,2549
...,...,...,...,...,...,...,...,...,...,...,...
3133,56037.0,0,2,18,30,90,240,302,339,488,1024
3134,56039.0,0,20,96,100,129,335,430,577,802,1156
3135,56041.0,0,0,8,12,176,254,298,357,484,719
3136,56043.0,0,1,8,36,39,48,108,116,142,208


In [24]:

month_col

Index(['county', 'state', 'c3_data_id', 'index', 'February', 'March', 'April',
       'May', 'June', 'July', 'August', 'September', 'October', 'November'],
      dtype='object')

In [26]:
month_col = df.columns[1:]
for i in range(len(month_col)-1):
    df[month_col[i+1]] = mcases[month_col[i+1]] - mcases[month_col[i]]

df

,fips_code,February,March,April,May,June,July,August,September,October,November
0,1001.0,0,7,37,177,309,442,376,443,368,297
1,1003.0,0,19,155,116,373,2393,1366,1218,1300,706
2,1005.0,0,0,39,128,155,228,78,268,164,68
3,1007.0,0,3,39,33,92,188,180,129,209,113
4,1009.0,0,5,32,25,142,481,349,595,445,475
...,...,...,...,...,...,...,...,...,...,...,...
3133,56037.0,0,2,16,12,60,150,62,37,149,536
3134,56039.0,0,20,76,4,29,206,95,147,225,354
3135,56041.0,0,0,8,4,164,78,44,59,127,235
3136,56043.0,0,1,7,28,3,9,60,8,26,66


## Saving New Cases to CSV Format

In [29]:
# new cases
df.to_csv('./all_counties_model/all_monthly_new_cases.csv', index = False)
# cumulative cases
cumulative_cases.to_csv('./all_counties_model/all_monthly_cumulative_cases.csv', index = False)